In [50]:
from math import inf
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import pyemd
import nltk
from gensim.models import Word2Vec
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

model = Word2Vec.load('pt.bin')
model.init_sims(replace=True)

data_url="./results.csv"
data = pd.read_csv(data_url).replace(np.nan, '', regex=True)

[nltk_data] Downloading package punkt to /home/vinha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vinha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/vinha/workspace/w2v/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...
5,Rosângela uma das crianças sequestradas por f...,“Roubaram minha identidade” diz Rosângela Par...,Joana Oliveira,30/03/2019 22:21:08,Brasil,Rosângela Serra Paraná não sabe quantos anos t...,https://brasil.elpais.com/brasil/2019/03/29/po...
6,“Lógica de usar torturadores da ditadura no cr...,Aloy Jupiara coautor de 'Os porões da contrav...,Felipe Betim,01/04/2019 13:22:17,Brasil,Trajetória similar tiveram outros vários agent...,https://brasil.elpais.com/brasil/2019/03/29/po...
7,Bolsonaro troca embaixada por escritório em Je...,Autoridade Palestina convoca embaixador no Bra...,Juan Carlos Sanz,01/04/2019 12:26:32,Brasil,A devoção dos cristãos evangélicos em seu apoi...,https://brasil.elpais.com/brasil/2019/03/31/po...
8,O lado mais sombrio de Dickens,Escritor e jornalista tentou internar sua mulh...,Rafa de Miguel,10/03/2019 14:29:37,Cultura,O homem mais famoso da era vitoriana o “poeta...,https://brasil.elpais.com/brasil/2019/03/01/cu...
9,O amor entre mulheres que sacudiu o Paraguai,O drama ganhou três prêmios em Berlim e provoc...,Gregorio Belinchón,09/03/2019 12:10:23,Cultura,Depois de inclusive o da crítica internacion...,https://brasil.elpais.com/brasil/2019/03/07/cu...


## Remove Stopwords + pre processamento

In [98]:
def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    year_pattern = rpt(r'\d{4}')
    
    patterns = [word_pattern, year_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token.lower())
        words.extend(tokens)
    return words

noticia = " ".join(parse(data.title[32]))

## Implemente uma função que recebe uma notícia e retorna os vetores (word embeddings) das palavras do título dessa notícia a partir dos word embeddings pré-treinados com o gensim (30 pts).

In [99]:
df = pd.DataFrame(columns=noticia.split())
for word in noticia.split():
    df[word] = model[word]
df

/home/vinha/workspace/w2v/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,ética,solidariedade,feminista
0,-0.021368,0.011606,0.067082
1,-0.086606,-0.143849,-0.106003
2,-0.024290,-0.023459,-0.061608
3,0.130540,-0.001413,0.004430
4,-0.004264,0.005835,0.010109
5,-0.078285,-0.009561,-0.049150
6,0.072516,0.077436,0.001315
7,-0.033178,0.075304,0.079947
8,-0.065992,0.042702,-0.025590
9,-0.068537,0.056968,-0.033179


## Implemente uma função que calcula o WMD (Word Mover's Distance) entre duas notícias usando os embeddings das palavras dos respectivos títulos

In [140]:
def calc_wmd(doc1, doc2):
    words1 = doc1
    words2 = doc2
  
    wmd = 0
    for w1 in words1:
        min_dist = inf
        for w2 in words2:
            min_dist = min(min_dist, sqrt(sum((model[w1] - model[w2]) ** 2)))
        wmd += min_dist * 1.0/len(words1)
 
    return wmd
# Noticias nao similares
wmd = calc_wmd(parse(data.title[30]), parse(data.title[32]))
print(data.title[30])
print(data.title[32])
print("WMD: ", wmd)

Matando como em ‘Call of Duty’
A ética da solidariedade feminista
WMD:  1.4371360696829516


/home/vinha/workspace/w2v/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [141]:
# Noticias MUITO similares:

wmd = calc_wmd(parse(data.title[84]), parse(data.title[43]))
print(data.title[84])
print(data.title[43])
print("WMD: ", wmd)

Onde e como assistir a Real Madrid x Barcelona pelas semifinais da Copa do Rei
Onde e como assistir a Real Madrid x Barcelona pelo Campeonato Espanhol
WMD:  0.27940795748299774


/home/vinha/workspace/w2v/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


## Implemente uma função que possa receber qualquer notícia como entrada e retornar as top-3 notícias mais similares (menos distantes) a ela

In [156]:
def top3similar(doc):
    a = []
    b = []
    for new in data.title:
        try:
            wmd = calc_wmd(parse(doc), parse(new))
            a.append(wmd)
            b.append(new)
        except (KeyError, AssertionError):
            # some words were not found, what to do?
            pass
    res = []
    for i in range(min(3, len(res))):
        res.append(b[a.index(min(a))])
        del a[a.index(min(a))]
        del b[a.index(min(a))]
    return res
print("Search: ", data.title[43])
top3similar(data.title[43])

Search:  Onde e como assistir a Real Madrid x Barcelona pelo Campeonato Espanhol


/home/vinha/workspace/w2v/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


[]